In [1]:
import numpy as np
import xarray as xr
import xrlint.all as xrl

---

### XRLint - API Objects

In [2]:
xrl.core_plugin.meta

PluginMeta(name='_core', version='0.0.1')

In [3]:
list(xrl.core_plugin.rules.keys())

['dataset-title-attr', 'no-empty-attrs', 'var-units-attr']

In [4]:
xrl.core_plugin.rules['var-units-attr']

Rule(meta=RuleMeta(name='var-units-attr', version='1.0.0', description=None, docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.var_units_attr.VarUnitsAttr'>)

In [5]:
list(xrl.core_plugin.configs.keys())

['recommended']

In [6]:
list(xrl.formats.keys())

['html', 'json', 'markdown', 'simple']

In [7]:
xrl.formats['simple']

Formatter(meta=FormatterMeta(name='simple', version='1.0.0', schema=None), op_class=<class 'xrlint.formatters.simple.Simple'>)

---

### XRLint - Basic Usage

In [8]:
nx = 2
ny = 3
nt = 4
ds = xr.Dataset(
    attrs=dict(title="SST-Cimatology Subset"),
    coords={
        "x": xr.DataArray(
            np.linspace(1000, 2000, nx),
            dims="x", 
            attrs={"units": "m"}
        ),
        "y": xr.DataArray(
            np.linspace(1000, 2000, ny),
            dims="y", 
            attrs={"units": "m"}
        ),
        "time": xr.DataArray(
            [2010 + y for y in range(nt)], 
            dims="time", 
            attrs={"units": "years"}),
    },
    data_vars={
        "sst": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin"}
        ),
        "sst_anomaly": xr.DataArray(
            np.random.random((nt, ny, nx)), 
            dims=["time", "y", "x"], 
            attrs={"units": "kelvin"}
        )
    },
)

Pass the configuration of rules via `rules`. Pass a

- `dict[RuleSeveriry, RuleOptions]` to configure named rules,
- `list[str]` for multiple rule sets (not implemented yet),
- `str` to load a single prefined rule set (not implemented yet).

Here, `RuleSeverity` is either a

- `str`: one of `"error"`, `"warn"`, `"off"`
- `int`: one of `2` (error), `1` (warn), `0` (off)

and `RuleOptions` is a `dict[str, Any]`.

In [9]:
linter = xrl.Linter(
    plugins={
        xrl.core_plugin.meta.name: xrl.core_plugin
    }, 
    rules={
        "no-empty-attrs": "warn",
        "dataset-title-attr": "error",
        "var-units-attr": "warn"
    }
)

In [10]:
linter.verify_dataset(ds)

Result(config=Config(name=None, files=None, ignores=None, linter_options=None, opener_options=None, processor=None, plugins={'_core': Plugin(meta=PluginMeta(name='_core', version='0.0.1'), configs={'recommended': Config(name='recommended', files=None, ignores=None, linter_options=None, opener_options=None, processor=None, plugins=None, rules={'dataset-title-attr': RuleConfig(severity=2, args=(), kwargs={}), 'no-empty-attrs': RuleConfig(severity=2, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=2, args=(), kwargs={})}, settings=None)}, rules={'dataset-title-attr': Rule(meta=RuleMeta(name='dataset-title-attr', version='1.0.0', description=None, docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.dataset_title_attr.DatasetTitleAttr'>), 'no-empty-attrs': Rule(meta=RuleMeta(name='no-empty-attrs', version='1.0.0', description=None, docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.no_empty_attrs.NoEmptyAttrs'>), 'var-units-attr': Rule(meta=RuleMeta(name='var-units-attr', version='1.0.0', description=None, docs_url=None, schema=None, type='problem'), op_class=<class 'xrlint.plugins.core.rules.var_units_attr.VarUnitsAttr'>)}, processors={})}, rules={'no-empty-attrs': RuleConfig(severity=1, args=(), kwargs={}), 'dataset-title-attr': RuleConfig(severity=2, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=1, args=(), kwargs={})}, settings={'no-empty-attrs': RuleConfig(severity=1, args=(), kwargs={}), 'dataset-title-attr': RuleConfig(severity=2, args=(), kwargs={}), 'var-units-attr': RuleConfig(severity=1, args=(), kwargs={})}), file_path='<file>', messages=[], fixable_error_count=0, fixable_warning_count=0, error_count=0, fatal_error_count=0, warning_count=0)

In [11]:
invalid_ds = ds.copy()
invalid_ds.attrs = {}
invalid_ds.sst.attrs["units"] = 1

In [12]:
linter.verify_dataset(invalid_ds)

dataset.attrs,warn,Attributes are empty.,no-empty-attrs
dataset,error,Missing 'title' attribute in dataset.,dataset-title-attr
dataset.data_vars['sst'],warn,Invalid 'units' attribute in variable 'sst'.,var-units-attr


---

### XRLint - Custom Plugins

In [13]:
linter = xrl.Linter(
    plugins={
        "humpty-dumpty": xrl.core_plugin
    }, 
    rules={
        "humpty-dumpty/no-empty-attrs": "warn",
        "humpty-dumpty/dataset-title-attr": "error",
        "humpty-dumpty/var-units-attr": "warn"
    }
)

In [14]:
linter.verify_dataset(invalid_ds)

dataset.attrs,warn,Attributes are empty.,humpty-dumpty/no-empty-attrs
dataset,error,Missing 'title' attribute in dataset.,humpty-dumpty/dataset-title-attr
dataset.data_vars['sst'],warn,Invalid 'units' attribute in variable 'sst'.,humpty-dumpty/var-units-attr
